**bisect**

In [1]:
# 保持序列插入
import bisect
import random

my_list = []
for i in range(10):
    num = random.randint(1, 10)
    bisect.insort(my_list, num)
    print(num, my_list)

10 [10]
9 [9, 10]
4 [4, 9, 10]
4 [4, 4, 9, 10]
2 [2, 4, 4, 9, 10]
10 [2, 4, 4, 9, 10, 10]
4 [2, 4, 4, 4, 9, 10, 10]
1 [1, 2, 4, 4, 4, 9, 10, 10]
2 [1, 2, 2, 4, 4, 4, 9, 10, 10]
3 [1, 2, 2, 3, 4, 4, 4, 9, 10, 10]


**函数**

In [5]:
def func(a, b=2):
    pass

In [6]:
print(func.__code__.co_varnames)

('a', 'b')


In [13]:
# 函数注解
def func2(a: str, b: 'int>10' = 100) -> int:
    return b

In [12]:
func2.__annotations__

{'a': str, 'b': 'int>10', 'return': int}

In [16]:
from operator import mul
from functools import partial

print(mul(5, 2))
# partial 复制函数,并固定参数,使新函数变成传参更少的函数
mul2 = partial(mul, 5)
print(mul2(3))

10
15


**引用**
+ 垃圾回收:每个对象都会统计有多少引用指向自己。 当引用计数归零时， 对象立即就被销毁
+ del 不删除对象， 而是删除对象的引用

In [1]:
# Python对不可变类型施加的把戏，str、 bytes 和 frozenset 实例也有这种行为
t1 = (1, 2, 3)
t2 = tuple(t1)

In [2]:
t2 is t1 

True

In [3]:
t3 = t1[:]
t3 is t1

True

In [4]:
t1 = 'sss'
print(id(t1))
t2 = str(t1)
print(id(t2))

1464348489240
1464348489240


In [1]:
t1 = (1, 2, 3)
t2 = (1, 2, 3)
print(id(t1))
print(id(t2))
t1 = 'abc'
t2 = 'abc'
print(id(t1))
print(id(t2))

2384778880560
2384796720224
2384691657168
2384691657168


**鸭子类型**
+ 对象的类型无关紧要， 只要实现了特定的协议即可。

In [1]:
import collections
from random import shuffle

Card = collections.namedtuple('Card', ['rank', 'suit'])


class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self.cards = [
            Card(rank, suit) for suit in self.suits for rank in self.ranks
        ]

    def __len__(self):
        return len(self.cards)

    def __getitem__(self, position):
        return self.cards[position]


#     shuffle需要类有该方法
#     def __setitem__(self, key, value):
#         self.cards[key]=value


def set_cards(cls, key, value):
    cls.cards[key] = value

# 猴子补丁，属性在运行时的动态替换
FrenchDeck.__setitem__ = set_cards

deck = FrenchDeck()
shuffle(deck)
print(deck.cards[:5])

[Card(rank='10', suit='hearts'), Card(rank='K', suit='clubs'), Card(rank='7', suit='diamonds'), Card(rank='10', suit='clubs'), Card(rank='6', suit='diamonds')]


In [9]:
class T:
#     slots规定了类里的实例属性，定义不在内的属性将报错
    __slots__=('x','y')
    
    def __init__(self):
        self.x=1
        self.z=2
T()

AttributeError: 'T' object has no attribute 'z'

**十二  继承的优缺点**

In [6]:
import collections

# 内置类型的方法不会调用子类覆盖的方法
class A(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

a = A(one=1)
a

{'one': 1}

In [7]:
a['two']=2
a

{'one': 1, 'two': [2, 2]}

In [2]:
# 接子类化内置类型（如 dict、 list 或 str） 容易出错，因为内置类型的方法通常会忽略用户覆盖的方法。
# 不要子类化内置类型， 用户自己定义的类应该继承 collections 模块，如UserDict、 UserList 和 UserString
class A(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)


a = A(one=1)
print(a)

{'one': [1, 1]}


In [3]:
# 多重继承
class A:
    def ping(self):
        print('A-ping:', self)


class B(A):
    def pong(self):
        print('B-pong:', self)


class C(A):
    def pong(self):
        print('C-PONG:', self)


class D(B, C):
    def ping(self):
        super().ping()
        print('D-post-ping:', self)

    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)


d = D()
d.pingpong()
print(D.__mro__)

A-ping: <__main__.D object at 0x0000021AC796A6D8>
D-post-ping: <__main__.D object at 0x0000021AC796A6D8>
A-ping: <__main__.D object at 0x0000021AC796A6D8>
B-pong: <__main__.D object at 0x0000021AC796A6D8>
B-pong: <__main__.D object at 0x0000021AC796A6D8>
C-PONG: <__main__.D object at 0x0000021AC796A6D8>
(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)


**生成器迭代器**

+ 可迭代的对象：实现了`__iter__`方法，或者实现了`__getitem_`_方法，index从0开始
+ 迭代器：实现了`__next__`方法（可调用next()） 和` __iter__`方法(可被for)

In [11]:
class Sentence:
    def __init__(self):
        self.words = [1, 2, 3, 4]

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.words)

    def __getitem__(self, item):
        return self.words[item]

    # 返回迭代器
    def __iter__(self):
        return iter(self.words)


# 返回生成器
def iterate(cls):
    for word in cls.words:
        yield word
    return

In [12]:
print(iter(Sentence()))

# 猴子补丁动态替换
Sentence.__iter__ = iterate

print(iter(Sentence()))

<generator object iterate at 0x0000021AC7F30B88>


In [2]:
# iter(iterable) -> iterator
# iter(callable, sentinel) -> iterator

import random


def rd():
    return random.randint(1, 10)


rd_iter = iter(rd, 1)
for i in rd_iter:
    print(i)

2
8
4
9
8
4
9
6
7
